# Introduction              
              
In this project, you will explore the question of whether college education causally affects political participation. Specifically, you will use replication data from \href{https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1409483}{Who Matches? Propensity Scores and Bias in the Causal Eﬀects of Education on Participation} by former Berkeley PhD students John Henderson and Sara Chatfield. Their paper is itself a replication study of \href{https://www.jstor.org/stable/10.1017/s0022381608080651}{Reconsidering the Effects of Education on Political Participation} by Cindy Kam and Carl Palmer. In their original 2008 study, Kam and Palmer argue that college education has no effect on later political participation, and use the propensity score matching to show that pre-college political activity drives selection into college and later political participation. Henderson and Chatfield in their 2011 paper argue that the use of the propensity score matching in this context is inappropriate because of the bias that arises from small changes in the choice of variables used to model the propensity score. They use \href{http://sekhon.berkeley.edu/papers/GenMatch.pdf}{genetic matching} (at that point a new method), which uses an approach similar to optimal matching to optimize Mahalanobis distance weights. Even with genetic matching, they find that balance remains elusive however, thus leaving open the question of whether education causes political participation.              
              
You will use these data and debates to investigate the benefits and pitfalls associated with matching methods. Replication code for these papers is available online, but as you'll see, a lot has changed in the last decade or so of data science! Throughout the assignment, use tools we introduced in lab from the \href{https://www.tidyverse.org/}{tidyverse} and the \href{https://cran.r-project.org/web/packages/MatchIt/MatchIt.pdf}{MatchIt} packages. Specifically, try to use dplyr, tidyr, purrr, stringr, and ggplot instead of base R functions. While there are other matching software libraries available, MatchIt tends to be the most up to date and allows for consistent syntax.              
              
# Data              
              
The data is drawn from the \href{https://www.icpsr.umich.edu/web/ICPSR/studies/4023/datadocumentation#}{Youth-Parent Socialization Panel Study} which asked students and parents a variety of questions about their political participation. This survey was conducted in several waves. The first wave was in 1965 and established the baseline pre-treatment covariates. The treatment is whether the student attended college between 1965 and 1973 (the time when the next survey wave was administered). The outcome is an index that calculates the number of political activities the student engaged in after 1965. Specifically, the key variables in this study are:              
              
\begin{itemize}              
    \item \textbf{college}: Treatment of whether the student attended college or not. 1 if the student attended college between 1965 and 1973, 0 otherwise.              
    \item \textbf{ppnscal}: Outcome variable measuring the number of political activities the student participated in. Additive combination of whether the student voted in 1972 or 1980 (student\_vote), attended a campaign rally or meeting (student\_meeting), wore a campaign button (student\_button), donated money to a campaign (student\_money), communicated with an elected official (student\_communicate), attended a demonstration or protest (student\_demonstrate), was involved with a local community event (student\_community), or some other political participation (student\_other)              
\end{itemize}              
              
Otherwise, we also have covariates measured for survey responses to various questions about political attitudes. We have covariates measured for the students in the baseline year, covariates for their parents in the baseline year, and covariates from follow-up surveys. \textbf{Be careful here}. In general, post-treatment covariates will be clear from the name (i.e. student\_1973Married indicates whether the student was married in the 1973 survey). Be mindful that the baseline covariates were all measured in 1965, the treatment occurred between 1965 and 1973, and the outcomes are from 1973 and beyond. We will distribute the Appendix from Henderson and Chatfield that describes the covariates they used, but please reach out with any questions if you have questions about what a particular variable means.

In [1]:
# Load tidyverse and MatchIt
# Feel free to load other libraries as you wish
library(tidyverse)
library(MatchIt)

# Load ypsps data
ypsps <- read_csv('/home/mw/input/data3406/ypsps.csv')
head(ypsps)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Rows: 1254 Columns: 174

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (174): interviewid, college, student_vote, student_meeting, student_othe...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



interviewid,college,student_vote,student_meeting,student_other,student_button,student_money,student_communicate,student_demonstrate,student_community,⋯,student_1982meeting,student_1982other,student_1982button,student_1982money,student_1982communicate,student_1982demonstrate,student_1982community,student_1982IncSelf,student_1982HHInc,student_1982College
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,0,0,0,0,0,1,1,⋯,0,0,0,0,0,0,0,10,99,0
2,1,1,1,1,1,0,1,1,1,⋯,0,0,0,0,1,0,0,9,99,0
3,1,1,0,0,1,0,0,0,0,⋯,0,0,0,0,1,0,1,17,18,0
4,0,0,0,0,0,0,1,0,1,⋯,0,0,0,0,0,0,1,2,22,0
5,1,1,1,0,0,0,1,0,0,⋯,99,99,99,99,99,99,99,NA,NA,NA
6,1,1,0,0,0,1,0,0,0,⋯,1,1,1,1,0,0,1,14,21,1


# Randomization              
              
Matching is usually used in observational studies to to approximate random assignment to treatment. But could it be useful even in randomized studies? To explore the question do the following:              
              
\begin{enumerate}              
    \item Generate a vector that randomly assigns each unit to either treatment or control              
    \item Choose a baseline covariate (for either the student or parent). A binary covariate is probably best for this exercise.              
    \item Visualize the distribution of the covariate by treatment/control condition. Are treatment and control balanced on this covariate?              
    \item Simulate the first 3 steps 10,000 times and visualize the distribution of treatment/control balance across the simulations.              
\end{enumerate}

In [ ]:
# Generate a vector that randomly assigns each unit to treatment/control

# Choose a baseline covariate (use dplyr for this)

# Visualize the distribution by treatment/control (ggplot)

# Simulate this 10,000 times (monte carlo simulation - see R Refresher for a hint)

## Questions              
\begin{enumerate}              
    \item \textbf{What do you see across your simulations? Why does independence of treatment assignment and baseline covariates not guarantee balance of treatment assignment and baseline covariates?}              
\end{enumerate}              
              
\textbf{Your Answer}:               
              
# Propensity Score Matching              
              
## One Model              
Select covariates that you think best represent the "true" model predicting whether a student chooses to attend college, and estimate a propensity score model to calculate the Average Treatment Effect on the Treated (ATT). Plot the balance of the top 10 (or fewer if you select fewer covariates). Report the balance of the p-scores across both the treatment and control groups, and using a threshold of standardized mean difference of p-score $\leq .1$, report the number of covariates that meet that balance threshold.

In [ ]:
# Select covariates that represent the "true" model for selection, fit model

# Plot the balance for the top 10 covariates

# Report the overall balance and the proportion of covariates that meet the balance threshold

## Simulations              
              
Henderson/Chatfield argue that an improperly specified propensity score model can actually \textit{increase} the bias of the estimate. To demonstrate this, they simulate 800,000 different propensity score models by choosing different permutations of covariates. To investigate their claim, do the following:              
              
\begin{itemize}              
    \item Using as many simulations as is feasible (at least 10,000 should be ok, more is better!), randomly select the number of and the choice of covariates for the propensity score model.              
    \item For each run, store the ATT, the proportion of covariates that meet the standardized mean difference $\leq .1$ threshold, and the mean percent improvement in the standardized mean difference. You may also wish to store the entire models in a list and extract the relevant attributes as necessary.              
    \item Plot all of the ATTs against all of the balanced covariate proportions. You may randomly sample or use other techniques like transparency if you run into overplotting problems. Alternatively, you may use plots other than scatterplots, so long as you explore the relationship between ATT and the proportion of covariates that meet the balance threshold.              
    \item Finally choose 10 random models and plot their covariate balance plots (you may want to use a library like \href{https://cran.r-project.org/web/packages/gridExtra/index.html}{gridExtra} to arrange these)              
\end{itemize}              
              
\textbf{Note: There are lots of post-treatment covariates in this dataset (about 50!)! You need to be careful not to include these in the pre-treatment balancing. Many of you are probably used to selecting or dropping columns manually, or positionally. However, you may not always have a convenient arrangement of columns, nor is it fun to type out 50 different column names. Instead see if you can use dplyr 1.0.0 functions to programatically drop post-treatment variables (\href{https://www.tidyverse.org/blog/2020/03/dplyr-1-0-0-select-rename-relocate/}{here} is a useful tutorial).}

In [ ]:
# Remove post-treatment covariates

# Randomly select features

# Simulate random selection of features 10k+ times

# Fit p-score models and save ATTs, proportion of balanced covariates, and mean percent balance improvement

# Plot ATT v. proportion

# 10 random covariate balance plots (hint try gridExtra)
# Note: ggplot objects are finnicky so ask for help if you're struggling to automatically create them; consider using functions!

## Questions              
              
\begin{enumerate}              
    \item \textbf{How many simulations resulted in models with a higher proportion of balanced covariates? Do you have any concerns about this?}              
    \item \textbf{Your Answer}:              
    \item \textbf{Analyze the distribution of the ATTs. Do you have any concerns about this distribution?}              
    \item \textbf{Your Answer:}              
    \item \textbf{Do your 10 randomly chosen covariate balance plots produce similar numbers on the same covariates? Is it a concern if they do not?}              
    \item \textbf{Your Answer:}              
\end{enumerate}              
              
# Matching Algorithm of Your Choice              
              
## Simulate Alternative Model              
              
Henderson/Chatfield propose using genetic matching to learn the best weights for Mahalanobis distance matching. Choose a matching algorithm other than the propensity score (you may use genetic matching if you wish, but it is also fine to use the greedy or optimal algorithms we covered in lab instead). Repeat the same steps as specified in Section 4.2 and answer the following questions:

In [ ]:
# Remove post-treatment covariates

# Randomly select features

# Simulate random selection of features 10k+ times

# Fit  models and save ATTs, proportion of balanced covariates, and mean percent balance improvement

# Plot ATT v. proportion

# 10 random covariate balance plots (hint try gridExtra)
# Note: ggplot objects are finnicky so ask for help if you're struggling to automatically create them; consider using functions!

In [ ]:
# Visualization for distributions of percent improvement

## Questions              
              
\begin{enumerate}              
    \item \textbf{Does your alternative matching method have more runs with higher proportions of balanced covariates?}              
    \item \textbf{Your Answer:}              
    \item \textbf{Use a visualization to examine the change in the distribution of the percent improvement in balance in propensity score matching vs. the distribution of the percent improvement in balance in your new method. Which did better? Analyze the results in 1-2 sentences.}              
    \item \textbf{Your Answer:}              
\end{enumerate}              
              
\textbf{Optional:} Looking ahead to the discussion questions, you may choose to model the propensity score using an algorithm other than logistic regression and perform these simulations again, if you wish to explore the second discussion question further.              
              
# Discussion Questions              
              
\begin{enumerate}              
    \item Why might it be a good idea to do matching even if we have a randomized or as-if-random design?              
    \item \textbf{Your Answer:}              
    \item The standard way of estimating the propensity score is using a logistic regression to estimate probability of treatment. Given what we know about the curse of dimensionality, do you think there might be advantages to using other machine learning algorithms (decision trees, bagging/boosting forests, ensembles, etc.) to estimate propensity scores instead?              
    \item \textbf{Your Answer:}              
\end{enumerate}